## Code for Real time Drwosiness Det

In [1]:
import cv2
import numpy as np
import joblib
import tensorflow as tf
import mediapipe as mp

# Load Models
eye_model = tf.keras.models.load_model("G:/zolo/classroom-attention-monitor/models/eye_state_cnn.h5")
pose_model = joblib.load("G:/zolo/classroom-attention-monitor/models/headpose_model_fast.pkl")

# MediaPipe setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)
drawing = mp.solutions.drawing_utils

# Landmark indexes
LEFT_EYE = [33, 133, 160, 159, 158, 144, 153, 154, 155]
RIGHT_EYE = [362, 263, 387, 386, 385, 373, 380, 381, 382]

# Webcam
cap = cv2.VideoCapture(0)

# Drowsiness tracking
closed_frames = 0
CLOSED_THRESHOLD = 15  # consecutive frames
DROWSY = False

IMG_SIZE = 34  # input size for CNN

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    h, w, _ = frame.shape
    label = "Awake"
    color = (0, 255, 0)

    if results.multi_face_landmarks:
        face_landmarks = results.multi_face_landmarks[0]

        # Eye detection
        def extract_eye_image(eye_indices):
            points = [(int(face_landmarks.landmark[i].x * w),
                       int(face_landmarks.landmark[i].y * h)) for i in eye_indices]
            x_min = max(min(p[0] for p in points) - 5, 0)
            x_max = min(max(p[0] for p in points) + 5, w)
            y_min = max(min(p[1] for p in points) - 5, 0)
            y_max = min(max(p[1] for p in points) + 5, h)
            eye_img = frame[y_min:y_max, x_min:x_max]
            if eye_img.size == 0:
                return None
            eye_img = cv2.cvtColor(eye_img, cv2.COLOR_BGR2GRAY)
            eye_img = cv2.resize(eye_img, (IMG_SIZE, IMG_SIZE)) / 255.0
            return eye_img.reshape(1, IMG_SIZE, IMG_SIZE, 1)

        left_eye = extract_eye_image(LEFT_EYE)
        right_eye = extract_eye_image(RIGHT_EYE)

        eye_state = "Open"
        eyes_closed = False

        if left_eye is not None and right_eye is not None:
            left_pred = eye_model.predict(left_eye, verbose=0)[0]
            right_pred = eye_model.predict(right_eye, verbose=0)[0]
            left_state = np.argmax(left_pred)
            right_state = np.argmax(right_pred)

            if left_state == 1 and right_state == 1:
                closed_frames += 1
                eye_state = "Closed"
            else:
                closed_frames = 0

            # If closed for too long
            if closed_frames >= CLOSED_THRESHOLD:
                eyes_closed = True

        # Head pose detection
        pose_landmarks = []
        for i in range(50):
            lm = face_landmarks.landmark[i]
            pose_landmarks.extend([lm.x, lm.y])
        while len(pose_landmarks) < 100:
            pose_landmarks.append(0)

        yaw, pitch, roll = pose_model.predict(np.array(pose_landmarks).reshape(1, -1))[0]

        # Final drowsiness decision
        if eyes_closed or pitch > 15:
            label = "DROWSY"
            color = (0, 0, 255)
        else:
            label = f"Awake | Eyes: {eye_state} | Pitch: {pitch:.1f}"

        drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)

    # Display result
    cv2.putText(frame, label, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)
    cv2.imshow("Drowsiness Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\cam-env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegress